In [1]:
## Import Required Libraries

import kfp
import typing

In [2]:
from typing import NamedTuple
from kfp.components import *


def rf_model(file_name: str, n_estimators: int) -> NamedTuple('Outputs', [('Cf1', int), ('Cf2', int),
                                                                                     ('Cf3', int), ('Cf4', int)]):
        
    import pandas as pd
    import numpy as np
    
    df = pd.read_csv(file_name)
    
    return (1,2,3,4)

In [3]:
kfp_rf_model = kfp.components.func_to_container_op(func = rf_model, 
                                                          output_component_file = './rf-model-func.yaml',
                                                          packages_to_install = ['scikit-learn==0.22.2','numpy','pandas==1.0.3',
                                                                                 'imbalanced-learn==0.6.2','urllib3==1.24.2'])

In [5]:
import kfp.dsl as dsl

@dsl.pipeline(name='Read-Pipeline',description='ions using Random Forest Algorithm')
def Justread_func(file_name = "https://raw.githubusercontent.com/rujual/telco_churn/master/Data.csv", 
                n_estimators = 100):
    
    #Passing pipeline parameter and a constant value as operation arguments

    rf_model_task = kfp_rf_model(file_name, n_estimators = 100)

#For an operation with a single return value, the output reference can be accessed using `task.output` or `task.outputs['output_name']` syntax
#For an operation with a multiple return values, the output references can be accessed using `task.outputs['output_name']` syntax

In [6]:
pipeline_func = Justread_func
pipeline_filename = pipeline_func.__name__+'.pipeline.tar.gz'

import kfp.compiler as comp
comp.Compiler().compile(pipeline_func, pipeline_filename)

/home/ruju/anaconda3/lib/python3.7/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "Integer" based on the value "100".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
